# Klassifikation von Gedichten

## Der Datensatz 'Deutscher Lyrik-Korpus'

Wir nutzen den Datensatz [Deutscher Lyrik-Korpus](https://github.com/thomasnikolaushaider/DLK) von Thomas Nikolaus Haider aus folgender Publikation:

> Haider, T. and Eger, S. (2019, August). *Semantic Change and Emerging Tropes In a Large Corpus of New High German Poetry.* In Proceedings of the 1st International Workshop on Computational Approaches to Historical Language Change (pp. 216-222)

Dieser Datensatz umfasst 60821 Deutsche Gedichte aus mehreren Jahrhunderten:

![](img/poems_years.svg)

Aus diesem Datensatz haben wir im Verzeichnis `data` bereits

- 894 Gedichte von Heinrich Heine
- 872 Gedichte von Johann Wolfgang Goethe und
- 673 Gedichte von Kurt Tucholsky

ausgewählt.

Wir laden diese Auswahl mit [pandas](https://pandas.pydata.org) in einen [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html):

In [ ]:
import pandas as pd
EXTRACT = 'data/poems/selected_poems.json.bz2'
poems = pd.read_json(EXTRACT, compression='infer')
poems.head()

## Daten-Aufbereitung

### Schritt 1: Gedichte in One-Hot-kodierte Zeichenfolgen umwandeln

Wir bestimmen zuerst die Menge aller in den Gedichten auftretenden Zeichen:

In [ ]:
used_alphabet = set().union(*poems['text'].apply(set))
''.join(sorted(used_alphabet))

Wir werden uns auf einen kleineren Zeichensatz beschränken und ersetzen jedes Zeichen durch

- dessen Index in unserem Zeichensatz, beginnend bei 1, falls es enthalten ist, 
- eine 0, falls es nicht im Zeichensatz liegt:

In [ ]:
ALPHABET = 'abcdefghijklmnopqrstuvwxyzäöüßABCDEFGHIKLMNOPQRSTUVWXZYÄÖÜ .,;:!?-()"\'\n'
char_index = {char: index + 1 for index, char in enumerate(ALPHABET)}

def index_characters(text):
    return [char_index.get(char, 0) for char in text]
                                              
poems['characters'] = poems.text.apply(index_characters)
poems[['text', 'characters']].head()

Als nächstes ersetzen wir jeden Zeichen-Index durch den entsprechenden One-Hot-Kode:

In [ ]:
import numpy as np

eye = np.eye(len(ALPHABET))
zeros = np.zeros((1, len(ALPHABET)))
codes = np.vstack([zeros, eye])
codes.shape, codes

In [ ]:
poems['characters_ohe'] = poems.characters.apply(lambda chars: codes[chars])
poems['characters_ohe'].head()

Zum Schluss packen wir die Folgen der One-Hot-Kodes in eine Matrix und schneiden dabei die Gedichte auf eine feste Länge. Dazu verwenden wir die Hilfsfunktion [`pad_sequences`](https://keras.io/preprocessing/sequence/) von [Keras](https://keras.io):

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 1000
X = pad_sequences(poems.characters_ohe, maxlen=MAX_LEN)
X.shape

### Schritt 2: Autoren kodieren

Als nächstes kodieren wir die Label, also die Autoren. Das können wir ähnlich wie oben machen oder mit Hilfe der Funktion [`get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) von [Pandas](https://pandas.pydata.org):

In [ ]:
authors_ohe = pd.get_dummies(poems.author)
authors_ohe.head()
authors = authors_ohe.columns

Damit erhalten wir unsere Label wie folgt:

In [ ]:
y = authors_ohe.values
y.shape, y[:5]

### Schritt 3: Aufteilung in Trainings- und Testdaten

Als nächstes zerlegen wir unsere Daten in eine Trainings- und eine Test-Menge:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,  train_size=0.7)
X_train.shape, X_test.shape, y_train.shape

## Klassifikation mit einem neuronalen Netz

### Versuch 1: einfaches Feed-Forward-Netz

Probieren wir die Klassifikation mit einem einfachen Netz:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

def build_model():
    return Sequential([
        Flatten(),
        Dense(1024, activation='relu'),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(3, activation='softmax')
    ])

def train_model(model, epochs=5, batch_size=32):
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')
    history = model.fit(X_train,y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
    return model, pd.DataFrame(history.history)
    
model = build_model()
model, history = train_model(model)

Wir beobachten ein massives Over-Fitting.

### Versuch 2: Faltungsschichten

Als nächstes trainieren wir ein Faltungsnetz mit

- mehreren [**Faltungsschichten**](https://keras.io/layers/convolutional/) zur Muster-Extraktion und
- einer [**dichten Schicht**](https://keras.io/layers/core/) zur Klassifikation.

In [ ]:
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

def build_model():
    return Sequential([
        Conv1D(64, kernel_size=3, strides=1, activation='relu', input_shape=(MAX_LEN, len(ALPHABET))),
        Conv1D(128, kernel_size=3, strides=1, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')
    ])

model, history = train_model(build_model())

Visualisieren wir den Trainingsverlauf:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()

def plot_history(history):
    _, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5))
    history[['loss', 'val_loss']].plot.line(ax=ax1)
    history[['accuracy', 'val_accuracy']].plot.line(ax=ax2)

plot_history(history)

Werten wir schließlich das Modell auf den Testdaten aus:

In [ ]:
def validate(model):
    y_pred = np.argmax(model.predict(X_test), axis=1)
    y_true = np.argmax(y_test, axis=1)
    return y_true, y_pred

y_true, y_pred = validate(model)

Nun schauen wir uns die Konfusionsmatrix und ein paar Scores an:

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=authors))

In [ ]:
def confusion(y_true, y_pred):
    confusion_matrix = pd.crosstab(y_pred, y_true)
    confusion_matrix.index = pd.Index(authors, name="Vorhersage")
    confusion_matrix.columns = pd.Index(authors, name="Wahrheit")
    return confusion_matrix

confusion(y_true, y_pred)

Es ist nicht überraschend, dass Goethe und Heine eher miteinander verwechselt werden als mit Tucholsky...

## Aufgabe: Trainieren einer Einbettungs-Schicht

Wir können das Modell auch mit Zeichen-Index-Folgen statt mit Zeichen-One-Hot-Code-Folgen füttern und die [`Embedding`](https://keras.io/layers/embeddings/)-Schicht von Keras verwenden, um im Modell eine Zeichen-Einbettung zu trainieren.

In [ ]:
X = pad_sequences(poems.characters, maxlen=MAX_LEN)
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7)

Verwende nun als erste Schicht [`Embedding`](https://keras.io/layers/embeddings/) und füttere das Modell mit Zeichen-Index-Folgen:

In [ ]:
from tensorflow.keras.layers import Embedding, MaxPooling1D

def build_embedding_model():
# Your code here!
    pass

Schau, ob das Modell das tut, was es soll!

In [ ]:
model, _ = train_model(build_embedding_model(), epochs=10)
confusion(*validate(model))